Data Wrangling : We RateDogs Twitter Data

In [661]:
#import required libraries

import pandas as pd
import numpy as np
import requests
import os
import re
import json
import datetime
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

Data Gathering

In [662]:
#twitter archive
#reading csv file
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')


In [663]:
#image prediction
#download URL

url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
response = requests.get(url)


In [664]:
#get file name
file_name = url.split('/')[-1]
file_name

'image-predictions.tsv'

In [665]:
if not os.path.isfile(file_name):
    with open(file_name , mode="wb") as file:
       file.write(response.content)

In [666]:
#reading TSV file
image_predictions = pd.read_csv('image-predictions.tsv' , sep='\t')


In [667]:
#tweet data
df_list = []

with open('tweet-json.txt', 'r') as file:
    for line in file:
        tweet = json.loads(line)
        tweet_id = tweet['id']
        retweet_count = tweet['retweet_count']
        fav_count = tweet['favorite_count']
        df_list.append({'tweet_id':tweet_id,
                       'retweet_count': retweet_count,
                       'favorite_count': fav_count})

#load tweet data into pandas dataframe
tweet_data = pd.DataFrame(df_list)


Data Assessing

inspecting the dataset for:
- data quality issues
- tidiness issues

twitter archive

visual assessment

In [668]:
twitter_archive

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


programmatic assessment

In [669]:
twitter_archive.sample(10)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
766,777684233540206592,NaN,NaN,2016-09-19 01:42:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...","""Yep... just as I suspected. You're not flossi...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/777684233...,12,10,None,None,None,None,None
435,820446719150292993,NaN,NaN,2017-01-15 01:45:15 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Peaches. She's the ulti...,8.001414e+17,4.196984e+09,2016-11-20 00:59:15 +0000,https://twitter.com/dog_rates/status/800141422...,13,10,Peaches,None,None,None,None
2247,667873844930215936,NaN,NaN,2015-11-21 01:15:07 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Neat dog. Lots of spikes. Always in push-up po...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667873844...,10,10,None,None,None,None,None
1650,683515932363329536,NaN,NaN,2016-01-03 05:11:12 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",HEY PUP WHAT'S THE PART OF THE HUMAN BODY THAT...,NaN,NaN,NaN,https://vine.co/v/ibvnzrauFuV,11,10,None,None,None,None,None
2234,668190681446379520,NaN,NaN,2015-11-21 22:14:07 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Skittles. I would kidnap Skittles. Pin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/668190681...,12,10,Skittles,None,None,None,None
346,831926988323639298,8.319030e+17,2.068372e+07,2017-02-15 18:03:45 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@UNC can confirm 12/10,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
1857,675517828909424640,NaN,NaN,2015-12-12 03:29:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",12/10 stay woke https://t.co/XDiQw4Akiw,NaN,NaN,NaN,https://twitter.com/dog_rates/status/675517828...,12,10,None,None,None,None,None
629,794355576146903043,NaN,NaN,2016-11-04 01:48:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Butter. She can have wh...,7.887659e+17,4.196984e+09,2016-10-19 15:37:03 +0000,https://twitter.com/dog_rates/status/788765914...,12,10,Butter,None,None,None,None
1653,683462770029932544,NaN,NaN,2016-01-03 01:39:57 +0000,"<a href=""http://twitter.com/download/iphone"" r...","""Hello forest pupper I am house pupper welcome...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/683462770...,8,10,None,None,None,pupper,None
1330,705786532653883392,7.032559e+17,4.196984e+09,2016-03-04 16:06:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...","Seriously, add us 🐶 11/10 for sad wet pupper h...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/705786532...,11,10,None,None,None,pupper,None


In [670]:
twitter_archive.shape

(2356, 17)

In [671]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [672]:
twitter_archive[['rating_numerator','rating_denominator']].describe()

,rating_numerator,rating_denominator
count,2356.000000,2356.000000
mean,13.126486,10.455433
std,45.876648,6.745237
min,0.000000,0.000000
25%,10.000000,10.000000
50%,11.000000,10.000000
75%,12.000000,10.000000
max,1776.000000,170.000000


In [673]:
#checking for duplicates
twitter_archive.duplicated().sum()

0

In [674]:
twitter_archive[[ 'doggo', 'floofer', 'pupper', 'puppo']]

,doggo,floofer,pupper,puppo
0,None,None,None,None
1,None,None,None,None
2,None,None,None,None
3,None,None,None,None
4,None,None,None,None
5,None,None,None,None
6,None,None,None,None
7,None,None,None,None
8,None,None,None,None
9,doggo,None,None,None


In [675]:
twitter_archive['name'].value_counts()

None         745
a             55
Charlie       12
Cooper        11
Lucy          11
Oliver        11
Lola          10
Penny         10
Tucker        10
Bo             9
Winston        9
the            8
Sadie          8
Bailey         7
an             7
Buddy          7
Daisy          7
Toby           7
Koda           6
Scout          6
Leo            6
Oscar          6
Jack           6
Bella          6
Stanley        6
Jax            6
Milo           6
Rusty          6
Dave           6
George         5
            ... 
Ole            1
Jonah          1
Socks          1
Beebop         1
Miguel         1
Malikai        1
Dotsy          1
Dallas         1
Sully          1
Stark          1
Oddie          1
Timber         1
Karma          1
Trevith        1
Aja            1
Alejandro      1
Jimbo          1
Berb           1
Fiji           1
Tiger          1
Lacy           1
Berkeley       1
Laika          1
Brandi         1
Binky          1
Ralph          1
Jed            1
Norman        

image predictions


visual assessment

In [676]:
image_predictions

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


programmatic assessment

In [677]:
image_predictions.sample(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
521,676575501977128964,https://pbs.twimg.com/media/CWOt07EUsAAnOYW.jpg,1,feather_boa,0.424106,False,Yorkshire_terrier,0.073144,True,Shetland_sheepdog,0.057598,True
1193,740214038584557568,https://pbs.twimg.com/media/CkXEu2OUoAAs8yU.jpg,1,Chesapeake_Bay_retriever,0.586414,True,Labrador_retriever,0.189782,True,vizsla,0.067607,True
1131,728046963732717569,https://pbs.twimg.com/media/ChqK2cVWMAAE5Zj.jpg,1,Newfoundland,0.255971,True,groenendael,0.175583,True,German_shepherd,0.164135,True
1797,831552930092285952,https://pbs.twimg.com/media/C4pE-I0WQAABveu.jpg,1,Chihuahua,0.257415,True,Pembroke,0.161442,True,French_bulldog,0.092143,True
1811,832998151111966721,https://pbs.twimg.com/media/C49nZavUYAEJjGw.jpg,1,boxer,0.539036,True,French_bulldog,0.317617,True,bull_mastiff,0.093928,True
291,671182547775299584,https://pbs.twimg.com/media/CVCE9uYXIAEtSzR.jpg,1,Rottweiler,0.331179,True,kelpie,0.218601,True,Appenzeller,0.182520,True
799,691321916024623104,https://pbs.twimg.com/media/CZgRmk0UcAAxeuQ.jpg,1,Rottweiler,0.508981,True,German_shepherd,0.207897,True,kelpie,0.094353,True
1450,776218204058357768,https://pbs.twimg.com/media/CsWuVEdWcAAqbe9.jpg,1,Samoyed,0.940326,True,Pomeranian,0.055527,True,keeshond,0.002226,True
560,677716515794329600,https://pbs.twimg.com/media/CWe7kw9W4AE8UJh.jpg,1,teddy,0.662908,False,crib,0.031891,False,chow,0.025438,True
688,684177701129875456,https://pbs.twimg.com/media/CX6v_JOWsAE0beZ.jpg,1,chow,0.334783,True,German_shepherd,0.162647,True,golden_retriever,0.138612,True


In [678]:
image_predictions.shape

(2075, 12)

In [679]:
image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [680]:
image_predictions[['p1_conf','p2_conf','p3_conf']].describe()

,p1_conf,p2_conf,p3_conf
count,2075.000000,2.075000e+03,2.075000e+03
mean,0.594548,1.345886e-01,6.032417e-02
std,0.271174,1.006657e-01,5.090593e-02
min,0.044333,1.011300e-08,1.740170e-10
25%,0.364412,5.388625e-02,1.622240e-02
50%,0.588230,1.181810e-01,4.944380e-02
75%,0.843855,1.955655e-01,9.180755e-02
max,1.000000,4.880140e-01,2.734190e-01


In [681]:
#checking for duplicates
image_predictions.duplicated().sum()


0

In [682]:
image_predictions.loc[(image_predictions['p1_dog'] == False) & 
                        (image_predictions['p2_dog'] == False) &
                        (image_predictions['p3_dog'] == False) ]

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,4.588540e-02,False,terrapin,1.788530e-02,False
17,666104133288665088,https://pbs.twimg.com/media/CT56LSZWoAAlJj2.jpg,1,hen,0.965932,False,cock,3.391940e-02,False,partridge,5.206580e-05,False
18,666268910803644416,https://pbs.twimg.com/media/CT8QCd1WEAADXws.jpg,1,desktop_computer,0.086502,False,desk,8.554740e-02,False,bookcase,7.947970e-02,False
21,666293911632134144,https://pbs.twimg.com/media/CT8mx7KW4AEQu8N.jpg,1,three-toed_sloth,0.914671,False,otter,1.525000e-02,False,great_grey_owl,1.320720e-02,False
25,666362758909284353,https://pbs.twimg.com/media/CT9lXGsUcAAyUFt.jpg,1,guinea_pig,0.996496,False,skunk,2.402450e-03,False,hamster,4.608630e-04,False
29,666411507551481857,https://pbs.twimg.com/media/CT-RugiWIAELEaq.jpg,1,coho,0.404640,False,barracouta,2.714850e-01,False,gar,1.899450e-01,False
45,666786068205871104,https://pbs.twimg.com/media/CUDmZIkWcAAIPPe.jpg,1,snail,0.999888,False,slug,5.514170e-05,False,acorn,2.625800e-05,False
50,666837028449972224,https://pbs.twimg.com/media/CUEUva1WsAA2jPb.jpg,1,triceratops,0.442113,False,armadillo,1.140710e-01,False,common_iguana,4.325530e-02,False
51,666983947667116034,https://pbs.twimg.com/media/CUGaXDhW4AY9JUH.jpg,1,swab,0.589446,False,chain_saw,1.901420e-01,False,wig,3.450970e-02,False
53,667012601033924608,https://pbs.twimg.com/media/CUG0bC0U8AAw2su.jpg,1,hyena,0.987230,False,African_hunting_dog,1.260080e-02,False,coyote,5.735010e-05,False


In [683]:
image_predictions['p1'].value_counts()

golden_retriever             150
Labrador_retriever           100
Pembroke                      89
Chihuahua                     83
pug                           57
chow                          44
Samoyed                       43
toy_poodle                    39
Pomeranian                    38
malamute                      30
cocker_spaniel                30
French_bulldog                26
Chesapeake_Bay_retriever      23
miniature_pinscher            23
seat_belt                     22
Staffordshire_bullterrier     20
Siberian_husky                20
German_shepherd               20
web_site                      19
Cardigan                      19
Maltese_dog                   18
Eskimo_dog                    18
beagle                        18
Shetland_sheepdog             18
teddy                         18
Shih-Tzu                      17
Lakeland_terrier              17
Rottweiler                    17
Italian_greyhound             16
kuvasz                        16
          

In [684]:
image_predictions['p2'].value_counts()

Labrador_retriever                104
golden_retriever                   92
Cardigan                           73
Chihuahua                          44
Pomeranian                         42
Chesapeake_Bay_retriever           41
French_bulldog                     41
toy_poodle                         37
cocker_spaniel                     34
miniature_poodle                   33
Siberian_husky                     33
beagle                             28
Eskimo_dog                         27
collie                             27
Pembroke                           27
kuvasz                             26
Italian_greyhound                  22
Pekinese                           21
American_Staffordshire_terrier     21
toy_terrier                        20
malinois                           20
Samoyed                            20
miniature_pinscher                 20
chow                               20
Boston_bull                        19
Norwegian_elkhound                 19
Staffordshir

In [685]:
image_predictions['p3'].value_counts()

Labrador_retriever                79
Chihuahua                         58
golden_retriever                  48
Eskimo_dog                        38
kelpie                            35
kuvasz                            34
Staffordshire_bullterrier         32
chow                              32
cocker_spaniel                    31
beagle                            31
Pomeranian                        29
Pekinese                          29
toy_poodle                        29
Chesapeake_Bay_retriever          27
Great_Pyrenees                    27
Pembroke                          27
malamute                          26
French_bulldog                    26
American_Staffordshire_terrier    24
Cardigan                          23
pug                               23
basenji                           21
bull_mastiff                      20
toy_terrier                       20
Siberian_husky                    19
Boston_bull                       17
Shetland_sheepdog                 17
L

In [686]:
image_predictions['img_num'].value_counts()

1    1780
2     198
3      66
4      31
Name: img_num, dtype: int64

tweet data 

visual assessment

In [687]:
tweet_data

,favorite_count,retweet_count,tweet_id
0,39467,8853,892420643555336193
1,33819,6514,892177421306343426
2,25461,4328,891815181378084864
3,42908,8964,891689557279858688
4,41048,9774,891327558926688256
5,20562,3261,891087950875897856
6,12041,2158,890971913173991426
7,56848,16716,890729181411237888
8,28226,4429,890609185150312448
9,32467,7711,890240255349198849


programmatic assessment

In [688]:
tweet_data.sample(10)

,favorite_count,retweet_count,tweet_id
226,20229,4037,848324959059550208
2349,111,41,666049248165822465
787,24167,6478,774314403806253056
1943,1177,434,673707060090052608
1376,3080,1055,701570477911896070
1956,882,297,673580926094458881
1851,693,106,675706639471788032
817,8130,2013,770655142660169732
1525,2925,1006,690360449368465409
1222,2480,990,714214115368108032


In [689]:
tweet_data.shape

(2354, 3)

In [690]:
tweet_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
favorite_count    2354 non-null int64
retweet_count     2354 non-null int64
tweet_id          2354 non-null int64
dtypes: int64(3)
memory usage: 55.2 KB


In [691]:
tweet_data[['favorite_count','retweet_count']].describe()

,favorite_count,retweet_count
count,2354.000000,2354.000000
mean,8080.968564,3164.797366
std,11814.771334,5284.770364
min,0.000000,0.000000
25%,1415.000000,624.500000
50%,3603.500000,1473.500000
75%,10122.250000,3652.000000
max,132810.000000,79515.000000


In [692]:
#cheking for duplicates
tweet_data.duplicated().sum()

0

Data Assessment Summary 

- Quality Aspects:

1- text column has the link for the tweets and ratings at the end we can remove    it.

2- timestamp column dtype is str instead of datetime. 

3- rating_numerator and rating_denominator columns dtype are int instead of        float.

4- tweet_id column dtype is int instead if str.

5- We need only the tweets not the retweets, so we should remove the retweets      from the table.

6- We need only the tweets not the replys to them, so we should remove the        replys from the table.

7- some rating_denominator values do not equal 10.

8- columns (doggo, floofer, pupper, puppo) has None for missing values.

9- the rows in sources column are over 95% duplicated.

10- many predictions contain three false in image_predictions dataframe, which     means they are useless prediction data.

11- names in name columns are in lowercase

12- there are many wrong names in name column.

13- many tweets have no prediction image, they must be removed.


- Tidiness Aspects:

1- img_num, source, expanded_urls column is not important for analysis, so it's    better to be removed.

2- (doggo, floofer, pupper, puppo) columns are all for one thing which is dog      stage.

3- combine columns (p1,p2,p3) in column dog_type and columns
   (p1_conf,p2_conf,p3_conf) combine them in column p_confidence.
   
4- combine all datasets in one dataframe.


Data Cleaning

In [693]:
#take a copy of our data to start cleaning on it.
twitter_archive_clean = twitter_archive.copy()
image_predictions_clean = image_predictions.copy()
tweet_data_clean = tweet_data.copy()

Quality

define


remove the link of tweets and Rating from text column using extract.


In [694]:
#code
twitter_archive_clean['text']

0       This is Phineas. He's a mystical boy. Only eve...
1       This is Tilly. She's just checking pup on you....
2       This is Archie. He is a rare Norwegian Pouncin...
3       This is Darla. She commenced a snooze mid meal...
4       This is Franklin. He would like you to stop ca...
5       Here we have a majestic great white breaching ...
6       Meet Jax. He enjoys ice cream so much he gets ...
7       When you watch your owner call another dog a g...
8       This is Zoey. She doesn't want to be one of th...
9       This is Cassie. She is a college pup. Studying...
10      This is Koda. He is a South Australian decksha...
11      This is Bruno. He is a service shark. Only get...
12      Here's a puppo that seems to be on the fence a...
13      This is Ted. He does his best. Sometimes that'...
14      This is Stuart. He's sporting his favorite fan...
15      This is Oliver. You're witnessing one of his m...
16      This is Jim. He found a fren. Taught him how t...
17      This i

In [695]:
twitter_archive_clean['text'] = twitter_archive_clean['text'].str.extract('(.+(?=\s\d+/\d+\s))')

In [696]:
#test
twitter_archive_clean['text'].sample(5)

1162     This is Reginald. He starts screaming at random.
2303    This is Carll. He wants to be a donkey. But al...
1467    Meet Grady. He's very hungry. Too bad no one c...
1064    This is Leo. He's a vape god. Blows o's for da...
550     This is Django. He accidentally opened the fro...
Name: text, dtype: object

define

change dtype of timestamp from str to datetime

In [697]:
#code
twitter_archive_clean['timestamp'] = pd.to_datetime(twitter_archive_clean['timestamp'])


In [698]:
#test
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null datetime64[ns]
source                        2356 non-null object
text                          2234 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: datetime64[ns](1

define 

change type of rating_numerator and rating_denominator from int to float using astype.

In [699]:
#code
twitter_archive_clean['rating_numerator'] = twitter_archive_clean['rating_numerator'].astype(float)
twitter_archive_clean['rating_denominator'] = twitter_archive_clean['rating_denominator'].astype(float)

In [700]:
#test
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null datetime64[ns]
source                        2356 non-null object
text                          2234 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null float64
rating_denominator            2356 non-null float64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: datetime64[n

define 

change dtype of tweet_id in all dataframes from int to str using astype.

In [701]:
#code
twitter_archive_clean['tweet_id'] = twitter_archive_clean['tweet_id'].astype(str)
image_predictions_clean['tweet_id'] = image_predictions_clean['tweet_id'].astype(str)
tweet_data_clean['tweet_id'] = tweet_data_clean['tweet_id'].astype(str)

In [702]:
#test
twitter_archive_clean['tweet_id'].dtype


dtype('O')

In [703]:
#test
image_predictions_clean['tweet_id'].dtype

dtype('O')

In [704]:
#test
tweet_data_clean['tweet_id'].dtype

dtype('O')

define

delete rows that are retweets and replyies not an original tweets.

delete rows that have no images from both tweet_data and twitter_archive dataframes.

drop the columns (in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp).

delete tweets that are not in tweet_data from both twitter_archive and image_predictions dataframes.

In [705]:
#code


#creating a list of tweet_ids with images "tweets_with_image" and confirming its length
tweets_with_image = list(image_predictions_clean['tweet_id'].unique())

#confirming that all the tweets with images exist in the archive dataset
len(tweets_with_image) == twitter_archive_clean['tweet_id'].isin(tweets_with_image).sum()

True

In [706]:
#cleaning in action
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean['tweet_id'].isin(tweets_with_image)]

In [707]:
# extracting the tweets that include data in the retweet_status_id (extracting rows with retweets)
retweet_entries = twitter_archive_clean['retweeted_status_id'].notnull()
twitter_archive_clean[retweet_entries].shape

(81, 17)

In [708]:
# dropping the retweets from the archive data set
twitter_archive_clean = twitter_archive_clean[~retweet_entries]

In [709]:
# extracting replies entries with the same method as the retweets
reply_entries = twitter_archive_clean['in_reply_to_status_id'].notnull()
twitter_archive_clean[reply_entries].shape

(23, 17)

In [710]:
# dropping the replies from the archive data set
twitter_archive_clean = twitter_archive_clean[~reply_entries]

In [711]:
# now we check the image_prediction table for extra tweet ids not in the archive table
sum(np.logical_not(image_predictions_clean['tweet_id'].isin(list(twitter_archive_clean['tweet_id']))))


# dropping the retweets and replies ids from the image prediction dataframe
image_predictions_clean =image_predictions_clean[~np.logical_not(image_predictions_clean['tweet_id'].isin(list(twitter_archive_clean['tweet_id'])))]

In [712]:
# Dropping the retweets and replies ids from the api dataframe
tweet_data_clean = tweet_data_clean[~np.logical_not(tweet_data_clean['tweet_id'].isin(list(twitter_archive_clean['tweet_id'])))]


# Dropping the tweets that doesn't exist in the api table from both archieve and image tables
image_predictions_clean = image_predictions_clean[~np.logical_not(image_predictions_clean['tweet_id'].isin(list(tweet_data_clean['tweet_id'])))]
twitter_archive_clean = twitter_archive_clean[~np.logical_not(twitter_archive_clean['tweet_id'].isin(list(tweet_data_clean['tweet_id'])))]

In [713]:
# Dropping the unwanted columns of the Archive dataframe
twitter_archive_clean.drop(['in_reply_to_status_id','in_reply_to_user_id', 'retweeted_status_id'
                    , 'retweeted_status_user_id', 'retweeted_status_timestamp'], axis =1 ,inplace = True)

In [714]:
#test

twitter_archive_clean.shape

(1971, 12)

In [715]:
#test

image_predictions_clean.shape

(1971, 12)

In [716]:
#test

tweet_data.shape

(2354, 3)

In [717]:
twitter_archive_clean.columns

Index(['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls',
       'rating_numerator', 'rating_denominator', 'name', 'doggo', 'floofer',
       'pupper', 'puppo'],
      dtype='object')

define

make all values in name column uppercase using title.

In [718]:
#code
twitter_archive_clean['name']= twitter_archive_clean['name'].str.title()

In [719]:
#test
twitter_archive_clean['name'].sample(5)

1884    Tedders
1152     Lillie
1828       None
15       Oliver
1160     Luther
Name: name, dtype: object

define

replace wrong names in name column with nan values. 

In [720]:
#code

# List of extracted wring names
wrong_names = ["A", "An", "The", "Such","None"]
for i in wrong_names:
    twitter_archive_clean['name'].replace(i, np.nan, inplace = True)

In [721]:
#test

twitter_archive_clean.query('name in ["A", "An", "The", "Such","None"]')

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [722]:
#code
not_ten_denomi = list(twitter_archive_clean.query("rating_denominator != 10").index)
twitter_archive_clean.drop(index=not_ten_denomi, inplace=True)

In [723]:
#test
twitter_archive_clean.query("rating_denominator != 10").index

Int64Index([], dtype='int64')

define

set nan for missing values instead of none in columns doggo, floofer, pupper, puppo using replace.

In [724]:
#code

# Creating a list for column names
none_list = ['doggo', 'floofer', 'pupper', 'puppo']
# replacing each column 'None' values with nan
for name in none_list:
    twitter_archive_clean[name].replace("None",np.nan, inplace=True)


In [725]:
#test
for name in none_list:
    print(twitter_archive_clean[name].isnull().sum())

1881
1946
1745
1931


define

find the rows which have three false in image_predictions dataframe,
then drop them.


In [726]:
#code

three_false = image_predictions_clean.query("p1_dog==False and p2_dog==False and p3_dog == False").index
three_false

Int64Index([   6,   17,   18,   21,   25,   29,   45,   50,   51,   53,
            ...
            1953, 1956, 1975, 1979, 2012, 2021, 2022, 2046, 2052, 2074],
           dtype='int64', length=305)

In [727]:
image_predictions_clean.drop(index=three_false,inplace=True)

In [728]:
#test

image_predictions_clean.query("p1_dog==False and p2_dog==False and p3_dog == False").index

Int64Index([], dtype='int64')

Tidiness

define

drop img_num column

In [729]:
#code
image_predictions_clean.drop('img_num',axis=1,inplace=True)

In [730]:
#test
list(image_predictions_clean)

['tweet_id',
 'jpg_url',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog']

define

drop source , expanded_urls columns 

In [731]:
#code

twitter_archive_clean.drop(['source','expanded_urls'],axis=1,inplace=True)


In [732]:
#test
twitter_archive_clean.columns

Index(['tweet_id', 'timestamp', 'text', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

define

combine (doggo, floofer, pupper, puppo) columns in one column called 'dog_stage',
then drop the four columns.

In [733]:
#code

# List of the 4 columns names
dogs_stages = ['doggo', 'floofer', 'pupper', 'puppo']
# Making the dog_stage column by joining the all 4 columns with each other and seprating them by ', '
# after dropping the null values
twitter_archive_clean['dog_stage'] = twitter_archive_clean[dogs_stages].apply(lambda x: ', '.join(x.dropna().astype(str)),axis =1)

In [734]:
# Replacing the empty values with nan
twitter_archive_clean['dog_stage'] = twitter_archive_clean['dog_stage'].replace('', np.nan)


In [735]:
# drop the unwanted 4 columns
twitter_archive_clean = twitter_archive_clean.drop(columns = dogs_stages, axis =1)

In [736]:
#test
twitter_archive_clean['dog_stage'].value_counts()

pupper            201
doggo              63
puppo              22
doggo, pupper       8
floofer             7
doggo, puppo        1
doggo, floofer      1
Name: dog_stage, dtype: int64

In [737]:
#test
twitter_archive_clean.columns

Index(['tweet_id', 'timestamp', 'text', 'rating_numerator',
       'rating_denominator', 'name', 'dog_stage'],
      dtype='object')

In [738]:
tweet_data_clean = pd.DataFrame(columns=["tweet_id", "retweet_count", "favorite_count"])
twitter_archive_clean = pd.merge(twitter_archive_clean, tweet_data_clean , on=['tweet_id'] , how='left')

In [739]:
list(twitter_archive_clean)

['tweet_id',
 'timestamp',
 'text',
 'rating_numerator',
 'rating_denominator',
 'name',
 'dog_stage',
 'retweet_count',
 'favorite_count']

In [740]:
twitter_archive_clean.sample(10)

,tweet_id,timestamp,text,rating_numerator,rating_denominator,name,dog_stage,retweet_count,favorite_count
1415,677547928504967168,2015-12-17 17:56:29,Not much to say here. I just think everyone ne...,12.0,10.0,NaN,NaN,NaN,NaN
133,859607811541651456,2017-05-03 03:17:27,Sorry for the lack of posts today. I came home...,13.0,10.0,NaN,puppo,NaN,NaN
795,739932936087216128,2016-06-06 21:32:13,Say hello to Rorie. She's zen af. Just enjoyin...,10.0,10.0,Rorie,NaN,NaN,NaN
1092,698710712454139905,2016-02-14 03:29:49,This is Cuddles. He's not entirely sure how do...,10.0,10.0,Cuddles,NaN,NaN,NaN
873,724046343203856385,2016-04-24 01:24:35,This is Sarge. Not even he knows what his tong...,10.0,10.0,Sarge,NaN,NaN,NaN
645,759846353224826880,2016-07-31 20:21:02,This is Kirby. He's a Beneblip Cumberpat. Pret...,11.0,10.0,Kirby,NaN,NaN,NaN
1325,681320187870711809,2015-12-28 03:46:05,This is Glacier. He's a very happy pup. Loves ...,11.0,10.0,Glacier,NaN,NaN,NaN
393,808001312164028416,2016-12-11 17:31:39,This is Cooper. He likes to stick his tongue o...,12.0,10.0,Cooper,NaN,NaN,NaN
777,742465774154047488,2016-06-13 21:16:49,NaN,14.0,10.0,NaN,pupper,NaN,NaN
66,877556246731214848,2017-06-21 15:58:08,This is Penny. She's both pupset and fired pup...,12.0,10.0,Penny,NaN,NaN,NaN


In [741]:
twitter_archive_clean.columns

Index(['tweet_id', 'timestamp', 'text', 'rating_numerator',
       'rating_denominator', 'name', 'dog_stage', 'retweet_count',
       'favorite_count'],
      dtype='object')

define

create a column for image prediction and a column for confidence.
delete columns (p1,p2,p3) and (p1_conf,p2_conf,p3_conf)

In [742]:
#code

dog_type = []
p_confidence = []

def image(image_predictions_clean):
    if image_predictions_clean['p1_dog'] == True:
        dog_type.append(image_predictions_clean['p1'])
        p_confidence.append(image_predictions_clean['p1_conf'])
    elif image_predictions_clean['p2_dog'] == True:
        dog_type.append(image_predictions_clean['p2'])
        p_confidence.append(image_predictions_clean['p2_conf'])
    elif image_predictions_clean['p3_dog'] == True:
        dog_type.append(image_predictions_clean['p3'])
        p_confidence.append(image_predictions_clean['p3_conf'])
    else:
        dog_type.append('Error')
        p_confidence.append('Error')
        
image_predictions_clean.apply(image,axis=1)

image_predictions_clean['dog_type'] = dog_type
image_predictions_clean['p_confidence'] = p_confidence

In [743]:
image_predictions_clean = image_predictions_clean[image_predictions_clean['dog_type'] != 'Error']

In [744]:
image_predictions_clean = image_predictions_clean.drop([ 'p1', 
                                                      'p1_conf', 'p1_dog', 
                                                      'p2', 'p2_conf', 
                                                      'p2_dog', 'p3', 
                                                      'p3_conf', 
                                                      'p3_dog'], axis =1)

In [745]:
#test 

image_predictions_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1666 entries, 0 to 2073
Data columns (total 4 columns):
tweet_id        1666 non-null object
jpg_url         1666 non-null object
dog_type        1666 non-null object
p_confidence    1666 non-null float64
dtypes: float64(1), object(3)
memory usage: 65.1+ KB


In [748]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1954 entries, 0 to 1953
Data columns (total 9 columns):
tweet_id              1954 non-null object
timestamp             1954 non-null datetime64[ns]
text                  1889 non-null object
rating_numerator      1954 non-null float64
rating_denominator    1954 non-null float64
name                  1374 non-null object
dog_stage             303 non-null object
retweet_count         0 non-null object
favorite_count        0 non-null object
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 152.7+ KB


In [749]:
tweet_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
favorite_count    2354 non-null int64
retweet_count     2354 non-null int64
tweet_id          2354 non-null int64
dtypes: int64(3)
memory usage: 55.2 KB


define 

combine the three dataframes into one dataframe called final_df using merge.

In [752]:
#code

final_df = pd.merge(twitter_archive_clean,image_predictions_clean,on='tweet_id', how="left")
final_df = pd.merge(final_df,tweet_data_clean,on='tweet_id', how="left")

In [753]:
#test

final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1954 entries, 0 to 1953
Data columns (total 14 columns):
tweet_id              1954 non-null object
timestamp             1954 non-null datetime64[ns]
text                  1889 non-null object
rating_numerator      1954 non-null float64
rating_denominator    1954 non-null float64
name                  1374 non-null object
dog_stage             303 non-null object
retweet_count_x       0 non-null object
favorite_count_x      0 non-null object
jpg_url               1650 non-null object
dog_type              1650 non-null object
p_confidence          1650 non-null float64
retweet_count_y       0 non-null object
favorite_count_y      0 non-null object
dtypes: datetime64[ns](1), float64(3), object(10)
memory usage: 229.0+ KB


Data Storing

In [658]:
#store the final dataframe into file called twitter_archive_master
final_df.to_csv('twitter_archive_master.csv',index = False)

In [660]:
df  = pd.read_csv('twitter_archive_master.csv')
df.sample(5)


ValueError: a must be greater than 0

Data Analysis